# Importing the libraries:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM,Dropout
from tensorflow.keras.layers import Dense

# Importing the dataset:

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')


## Preprocessing the dataset:

In [3]:
# dropping the NANs values in dataset:
train_df = train_df.dropna()

# get the independent/dependent features from the dataset:
X = train_df.iloc[:,:-1]
y = train_df.iloc[:,-1]

In [4]:
X

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [5]:
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64

In [6]:
# Vocabulary_size / dictionary_size:
Vocabulary_size = 5000

# Prepare OneHotRepresentation:

## Prepare the Corpus:

In [7]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
messages = X.copy()
messages.reset_index(inplace=True)
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [9]:
ps = PorterStemmer()
corpus = []

for i in range(0, len(X)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [10]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

## Create oneHotRepresentation

In [11]:
oneHotRepres_messages = [one_hot(words,Vocabulary_size) for words in corpus]
oneHotRepres_messages

[[3187, 1550, 2908, 1889, 1033, 4391, 2517, 656, 2145, 4503],
 [4078, 2260, 2741, 1642, 1827, 4033, 948],
 [375, 391, 1327, 3961],
 [420, 2507, 91, 3994, 130, 1274],
 [1682, 1827, 3928, 127, 1400, 4590, 1827, 1943, 4876, 4434],
 [1890,
  302,
  697,
  360,
  634,
  1676,
  2764,
  80,
  1622,
  231,
  4591,
  802,
  3557,
  316,
  948],
 [720, 3891, 1124, 2876, 1585, 1828, 2576, 184, 3172, 3497, 4112],
 [4277, 1092, 4814, 3225, 339, 3499, 1676, 2421, 3172, 3497, 4112],
 [1243, 4065, 2367, 3264, 718, 762, 380, 1987, 1676, 4429],
 [2138, 3360, 4160, 2609, 2443, 4138, 3514, 1796],
 [143, 2485, 2702, 4640, 56, 3105, 296, 1865, 4607, 1546, 3775],
 [3994, 2565, 1033, 762, 1676, 339],
 [4531, 4153, 3800, 2325, 4344, 2426, 3089, 1053, 4197],
 [3302, 760, 812, 72, 2141, 1690, 975, 3172, 3497, 4112],
 [969, 1499, 3565, 3558, 1998, 3172, 3497, 4112],
 [559, 1091, 4120, 2849, 3229, 4131, 4436, 2666, 2749, 4953],
 [2586, 534, 2260],
 [2287, 3601, 4595, 2682, 1676, 3574, 1676, 948],
 [469, 4252, 274

# Prepare the padded_X from oneHotRepres_messages using pad_sequences:

In [12]:
fixed_title_sentence_len = 20

padded_X = pad_sequences(
    sequences=oneHotRepres_messages,
    padding='pre',
    maxlen=fixed_title_sentence_len,
)

In [13]:
padded_X

array([[   0,    0,    0, ...,  656, 2145, 4503],
       [   0,    0,    0, ..., 1827, 4033,  948],
       [   0,    0,    0, ...,  391, 1327, 3961],
       ...,
       [   0,    0,    0, ..., 3172, 3497, 4112],
       [   0,    0,    0, ..., 1817, 4907, 2443],
       [   0,    0,    0, ..., 3462, 3099, 1531]], dtype=int32)

# Prepare the Deeplearning Model:

In [14]:
total_features=40

model = Sequential()
model.add(Embedding(
    input_dim=Vocabulary_size,
    output_dim=total_features,
    input_length=fixed_title_sentence_len,
))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
# create X_final and y_final:

X_final = np.array(padded_X)
y_final = np.array(y)

In [16]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

# Splitting the Training/Testing dset split:

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.3, random_state=42)

In [18]:
X_train, y_train

(array([[   0,    0,    0, ..., 3994,  357,  736],
        [   0,    0,    0, ..., 1676,  887,  948],
        [   0,    0,    0, ..., 4646, 4274,  791],
        ...,
        [   0,    0,    0, ..., 3172, 3497, 4112],
        [   0,    0,    0, ..., 2553, 4775,  948],
        [   0,    0,    0, ..., 3928, 3677, 2847]], dtype=int32),
 array([1, 0, 1, ..., 0, 0, 1]))

In [19]:
X_test, y_test

(array([[   0,    0,    0, ..., 4355,  481, 3599],
        [   0,    0,    0, ..., 3172, 3497, 4112],
        [   0,    0,    0, ..., 3172, 3497, 4112],
        ...,
        [   0,    0,    0, ..., 4357, 2741, 1676],
        [   0,    0,    0, ...,  119, 1609,  948],
        [   0,    0,    0, ..., 3811, 4100,  833]], dtype=int32),
 array([1, 0, 0, ..., 1, 0, 1]))

# Traning the created DL model:

In [20]:
model.fit(
    X_train, y_train,
    validation_data=(X_test,y_test),
    epochs=50,
    batch_size=32)

Epoch 1/50
400/400 [==============================] - 11s 9ms/step - loss: 0.4089 - accuracy: 0.8009 - val_loss: 0.2018 - val_accuracy: 0.9114
Epoch 2/50
400/400 [==============================] - 3s 7ms/step - loss: 0.1367 - accuracy: 0.9475 - val_loss: 0.2057 - val_accuracy: 0.9192
Epoch 3/50
400/400 [==============================] - 3s 7ms/step - loss: 0.0994 - accuracy: 0.9628 - val_loss: 0.2292 - val_accuracy: 0.9191
Epoch 4/50
400/400 [==============================] - 3s 8ms/step - loss: 0.0704 - accuracy: 0.9763 - val_loss: 0.2420 - val_accuracy: 0.9191
Epoch 5/50
400/400 [==============================] - 3s 7ms/step - loss: 0.0515 - accuracy: 0.9839 - val_loss: 0.2888 - val_accuracy: 0.9162
Epoch 6/50
400/400 [==============================] - 3s 8ms/step - loss: 0.0370 - accuracy: 0.9882 - val_loss: 0.3419 - val_accuracy: 0.9109
Epoch 7/50
400/400 [==============================] - 3s 7ms/step - loss: 0.0236 - accuracy: 0.9929 - val_loss: 0.3515 - val_accuracy: 0.9123
Epoch

# Evaluating the Performance on the test dataset

In [21]:
y_pred = model.predict_classes(X_test)
y_pred

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[1],
       [0],
       [0],
       ...,
       [1],
       [0],
       [1]], dtype=int32)

In [28]:
from sklearn.metrics import confusion_matrix,accuracy_score

print(confusion_matrix(y_pred, y_test))

print("AccuracyScore on test Data: {:.2f}%".format(accuracy_score(y_pred,y_test)*100))


[[2830  205]
 [ 277 2174]]
AccuracyScore on test Data: 91.21%
